In [1]:
import os

In [4]:
%pwd

SyntaxError: invalid syntax (3668275833.py, line 1)

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\USER\\Documents\\GitHub\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    num_leaves: int
    learning_rate: float
    max_depth: int
    target_column: str

In [15]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LightGBM
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            n_estimators=params.n_estimators,
            num_leaves=params.num_leaves,
            learning_rate=params.learning_rate,
            max_depth=params.max_depth,
            target_column=schema.name
        )

        return model_trainer_config

In [17]:
import pandas as pd
import os
from mlFlowProject import logger
from lightgbm import LGBMClassifier
import joblib

In [20]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lgbm_clf = LGBMClassifier(n_estimators=self.config.n_estimators, num_leaves=self.config.num_leaves, learning_rate=self.config.learning_rate, max_depth= self.config.max_depth)
        lgbm_clf.fit(train_x, train_y)

        joblib.dump(lgbm_clf, os.path.join(self.config.root_dir, "lightGBM.joblib"))

In [21]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-05-25 13:50:17,321: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-25 13:50:17,324: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-25 13:50:17,330: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-25 13:50:17,332: INFO: common: created directory at: artifacts]
[2024-05-25 13:50:17,334: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\USER\Documents\GitHub\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC\venv\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\USER\Documents\GitHub\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC\venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1534, number of negative: 5967
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1385
[LightGBM] [Info] Number of data points in the train set: 7501, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.204506 -> initscore=-1.358366
[LightGBM] [Info] Start training from score -1.358366
